In [289]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import time
import math

In [290]:
torch.manual_seed(0)
np.random.seed(0)

In [291]:
input_window = 100 # number of input steps
output_window = 1 # number of prediction steps, in this model its fixed to one
block_len = input_window + output_window # for one input-output pair
batch_size = 10
train_size = 0.8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [292]:
def create_inout_sequences(input_data, input_window ,output_window):
    inout_seq = []
    L = len(input_data)
    block_num =  L - block_len + 1
    # total of [N - block_len + 1] blocks
    # where block_len = input_window + output_window

    for i in range( block_num ):
        train_seq = input_data[i : i + input_window]
        train_label = input_data[i + output_window : i + input_window + output_window]
        # print("train_label=", train_label)
        inout_seq.append((train_seq ,train_label))

    return torch.FloatTensor(np.array(inout_seq))

In [343]:
EXERCISE = "squat"
SLICE = "yes"
def get_data():
    # get the data
    angles = pd.read_csv("data/angles.csv")
    labels = pd.read_csv("data/labels.csv")

    # get data only for a single exercise
    if SLICE == "yes":
        labels = labels[labels["class"]==EXERCISE]
        angles = angles[angles["vid_id"].isin(labels["vid_id"])]

    # transform data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    angles1 = angles.drop(["vid_id","frame_order"], axis=1)
    angles2 = scaler.fit_transform(angles1)
    # angles2=angles
    print("len angles =",len(angles2))
    
    # make 80:20 split in train and test data
    samples = int(len(angles2) * train_size) # use a parameter to control training size
    print(samples)

    train_data = angles2[:samples]
    test_data = angles2[samples:]
    print(len(train_data))
    train_sequence = create_inout_sequences( train_data,input_window ,output_window)
    test_data = create_inout_sequences(test_data,input_window,output_window)
    print(train_sequence.shape)

    return train_sequence.to(device),test_data.to(device), scaler

train_data, val_data, scaler= get_data()

len angles = 11817
9453
9453
torch.Size([9353, 2, 100, 7])


In [344]:
len(train_data[0][0][-1])
scaler.inverse_transform(train_data[0][1])

array([[ 45.95745914,  81.40655851,  35.31698093, 118.72569764,
        145.04757687, 108.92687528, 110.06666011],
       [ 44.07924366,  79.32359956,  37.19674018, 103.62295977,
        137.77669768, 103.7313326 , 104.62871574],
       [ 34.98290598,  78.51498663,  41.72450673,  86.89696227,
        123.90273624,  93.21553859, 105.45896413],
       [ 30.29175084,  78.18402059,  39.90260294,  87.05849974,
        124.05389663,  87.08307928, 101.91383038],
       [ 21.01956678,  77.0753296 ,  40.87256018,  86.13374144,
        120.80011468,  61.59622844, 102.00630054],
       [ 37.24937281,  82.61837173,  37.53021555,  85.91353527,
        122.3634294 ,  94.45488108, 104.65253366],
       [ 33.53204606,  76.23924776,  41.23229977,  79.63009053,
        112.47110252,  93.66927818,  92.19466333],
       [ 35.76064817,  70.20299304,  39.07817938,  77.7497086 ,
        105.33941569, 105.75657502,  92.23103024],
       [ 34.72743573,  69.19033211,  34.96156806,  70.85173171,
         86.4706

In [295]:
# len(val_data[0][0])
# (val_data[0][1])

In [296]:
def get_batch(input_data, i, batch_size):
    batch_len = min(batch_size, len(input_data) - i)
    
    # Extract the input and target tensors from the batch
    data = input_data[i:i + batch_len]
    
    # Extracting the inputs and targets
    inputs = torch.stack([item[0] for item in data]).permute(1, 0, 2)  # Shape: [100, batch_len, 9]
    targets = torch.stack([item[1] for item in data]).permute(1, 0, 2)  # Shape: [100, batch_len, 9]
    
    return inputs, targets

In [297]:
class MyNN(nn.Module):
    
    def __init__(self, num_classes=7, input_size=7, hidden_size=64, num_layers=2):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linera_relu_stack = nn.Sequential(
            nn.Linear(7, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,7)
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linera_relu_stack(x)
        return logits
    
# model = LSTM().to(device)
# summary(model, (10,7))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Assuming the input has a sequence length of 10 and input size of 7
sequence_length = 10
input_size = 7
model = MyNN().to(device)
summary(model, (batch_size, sequence_length, input_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 10, 10, 512]           4,096
              ReLU-2          [-1, 10, 10, 512]               0
            Linear-3          [-1, 10, 10, 512]         262,656
              ReLU-4          [-1, 10, 10, 512]               0
            Linear-5            [-1, 10, 10, 7]           3,591
Total params: 270,343
Trainable params: 270,343
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.57
Params size (MB): 1.03
Estimated Total Size (MB): 2.60
----------------------------------------------------------------


In [298]:
for batch, i in enumerate(range(0, len(train_data), batch_size)):  # Now len-1 is not necessary
    # data and target are the same shape with (input_window,batch_len,1)
    data, targets = get_batch(train_data, i , batch_size)
    print(data.shape)
    break

torch.Size([100, 10, 7])


In [322]:
dummy_input = torch.randn(100,10,7)
pred = model(dummy_input)
# (pred.argmax(1) == y).type(torch.float).sum().item()

pred.argmax(1)[1]

tensor([[ 0.2907,  0.3137,  0.2512,  0.2176,  0.2039,  0.3596,  0.3431],
        [ 0.4960,  0.4223, -0.0091,  0.3137,  0.0999,  0.5591,  0.4925],
        [ 0.7441,  0.7393,  0.2833,  0.2641,  0.2324,  0.7450,  0.7131],
        [ 0.2590,  0.1392,  0.2351,  0.3388,  0.2816,  0.2953,  0.2948],
        [ 0.2597,  0.1787,  0.2314,  0.3713,  0.3636,  0.3318,  0.3435],
        [ 0.1076,  0.1642,  0.2959,  0.5887,  0.5596,  0.3688,  0.3069],
        [ 0.5681,  0.4148,  0.2414,  0.3416,  0.2998,  0.4355,  0.4008],
        [ 0.2004,  0.3365,  0.1990,  0.7395,  0.7002,  0.6139,  0.5765],
        [ 0.0184,  0.1498,  0.0972,  0.4392,  0.4903,  0.4561,  0.4614],
        [ 0.2927,  0.1936,  0.2136,  0.3943,  0.4047,  0.3308,  0.2385]],
       grad_fn=<SelectBackward0>)

In [325]:
def r_squared(y_true, y_pred):
    # Calculate the total sum of squares
    ss_total = torch.sum((y_true - torch.mean(y_true)) ** 2)
    
    # Calculate the residual sum of squares
    ss_residual = torch.sum((y_true - y_pred) ** 2)
    
    # Calculate R-squared
    r2 = 1 - (ss_residual / ss_total)
    
    return r2

In [332]:
# Get a batch
data, targets = get_batch(train_data, 10, batch_size)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [338]:

def train(model, train_data, optimizer, loss_fn, epoch):
    model.train()
    start_time = time.time()
    total_loss = 0
    for batch, i in enumerate(range(0, len(train_data), batch_size)):  # Now len-1 is not necessary
        # data and target are the same shape with (input_window,batch_len,1)
        data, targets = get_batch(train_data, i , batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [339]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        # for i in range(0, len(data_source) - 1, eval_batch_size): # Now len-1 is not necessary
        for i in range(0, len(data_source), eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0]) * loss_fn(output, targets).cpu().item()
    return total_loss / len(data_source)

In [340]:
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source , 0 , 1)
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])
            # (seq-len , batch-size , features-num)
            # input : [ m,m+1,...,m+n ] -> [m+1,...,m+n+1]
            data = torch.cat((data, output[-1:])) # [m,m+1,..., m+n+1]

    data = data.cpu().view(-1)

In [341]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

In [342]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data=train_data, optimizer=optimizer, loss_fn=loss_fn, epoch=epoch, model=model)
    if ( epoch % 5 == 0 ):
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
   
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 
    # using adam2

| epoch   1 |   187/  935 batches | lr 0.001000 | 11.31 ms | loss 0.01403 | ppl     1.01
| epoch   1 |   374/  935 batches | lr 0.001000 | 12.52 ms | loss 0.01351 | ppl     1.01
| epoch   1 |   561/  935 batches | lr 0.001000 | 11.52 ms | loss 0.01647 | ppl     1.02
| epoch   1 |   748/  935 batches | lr 0.001000 | 12.99 ms | loss 0.00659 | ppl     1.01
| epoch   1 |   935/  935 batches | lr 0.001000 | 11.98 ms | loss 0.00938 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 12.17s | valid loss 0.01422 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch   2 |   187/  935 batches | lr 0.000902 | 11.97 ms | loss 0.01432 | ppl     1.01
| epoch   2 |   374/  935 batches | lr 0.000902 | 12.18 ms | loss 0.01342 | ppl     1.01
| epoch   2 |   561/  935 batches | lr 0.000902 | 11.87 ms | loss 0.01641 | ppl     1.02
| epoch   2 |   748/  935 batche

In [352]:
data, targets = get_batch(train_data, 10 , batch_size)
scaler.inverse_transform(data[0])

array([[ 41.99378113,  77.24890775,  33.0099011 ,  64.01752334,
         76.9591175 , 110.00951515, 107.84009443],
       [ 47.91984599,  77.13379764,  33.08744455,  64.0421132 ,
         71.70387947, 108.94375274,  98.61781941],
       [ 48.81424244,  73.67586328,  30.70118199,  66.27497236,
         69.82400757, 106.34292627,  97.76381607],
       [ 75.53007608,  97.28785289,  26.32655197,  55.92125345,
         59.71817062, 118.08832456, 111.23514384],
       [ 70.86176239,  92.50072465,  25.91737583,  56.67867429,
         61.05501677, 114.38921129, 100.46428477],
       [ 61.14785789,  84.85567529,  26.45665497,  55.05013197,
         58.45584233, 111.25015183,  94.95421515],
       [ 81.53981118, 103.8554115 ,  24.37472707,  48.24288047,
         54.35841258, 118.4787551 , 110.10116985],
       [ 51.07430119,  73.87144832,  24.95333432,  53.18449559,
         55.47882851, 107.4377124 ,  84.65190794],
       [ 72.5018846 ,  90.18394647,  23.76135293,  53.49238085,
         55.0600

In [362]:
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source , 0 , 1)
    print(data[0])
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])
            print(output.shape)
            # (seq-len , batch-size , features-num)
            # input : [ m,m+1,...,m+n ] -> [m+1,...,m+n+1]
            data = torch.cat((data, output[-1:])) # [m,m+1,..., m+n+1]

    data = data.cpu().view(-1)

predict_future(model, val_data,100)

tensor([[0.1059, 0.1531, 0.2079, 0.2792, 0.2168, 0.0627, 0.7748]])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7])
torch.Size([100, 1, 7

In [ ]:
# # to be used for later use
# import os
# import pandas as pd
# from torchvision.io import read_image
# from torch.utils.data import Dataset

# class CustomImageDataset(Dataset):
#     def __init__(self, EXERCISE, annotations_file="data/labels.csv", data_file="data/angles.csv", SLICE="no", transform="yes"):
#         self.labels = pd.read_csv(annotations_file)
#         self.angles = pd.read_csv(data_file)
#         # self.data_file = data_file
#         if SLICE == "yes":
#             self.labels = self.labels[self.labels["class"]==EXERCISE]
#             self.angles = self.angles[self.angles["vid_id"].isin(self.labels["vid_id"])]
#         if transform == "yes":
#             scaler = MinMaxScaler(feature_range=(0, 1))
#             self.angles1 = self.angles.drop(["vid_id","frame_order"], axis=1)
#             self.tangles = scaler.fit_transform(self.angles1)

#     def __len__(self):
#         return len(self.angles)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.data_file, self.labels.iloc[idx, 0])
#         image = read_image(img_path)
#         label = self.labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label